In [ ]:
#pip install requests


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import time
import requests
import os
from pymongo import MongoClient
from pymongo.errors import PyMongoError
from dotenv import load_dotenv

# Cargar las variables del archivo .env
load_dotenv()

# Obtener los valores de MongoDB y la API_KEY desde las variables de entorno
MONGO_URI = os.getenv("MONGO_URI")
MONGO_DB = os.getenv("MONGO_DB")
MONGO_COLECCION = os.getenv("MONGO_COLECCION")
API_KEY = os.getenv("API_KEY")

# Configuración de los encabezados necesarios para la solicitud a la API
HEADERS = {
    "x-apikey": API_KEY,
    "accept": "application/json",
    "user-agent": "Tu User Agent Aquí",  # Aquí puedes colocar tu user agent o el que necesites
}

# Conexión a MongoDB usando las variables de entorno
client = MongoClient(MONGO_URI)  # Conectar con la URI proporcionada
db = client[MONGO_DB]  # Usar el nombre de la base de datos
coleccion = db[MONGO_COLECCION]  # Usar la colección

# Función para obtener datos de la API
def obtener_datos(Number):
    """
    Realiza una petición GET a la API con el número de la fila y obtiene los datos.
    """
    # Formar la URL con el número y agregar el sufijo "-1"
    url = f"https://www.brickeconomy.com/api/v1/set/{Number}-1"
    print(f"Obteniendo datos para el set: {Number}")  # Mensaje de depuración

    # Realizar la solicitud GET con los encabezados necesarios
    response = requests.get(url, headers=HEADERS)

    if response.status_code == 200:
        return response.json()  # Retorna los datos obtenidos si la solicitud fue exitosa
    else:
        print(f"Error al obtener datos para el número {Number}: {response.status_code}")
        return None

# Función para actualizar los datos en MongoDB
def actualizar_en_mongo(_id, nuevos_datos):
    """
    Actualiza el documento de MongoDB con los datos obtenidos de la API.
    """
    try:
        print(f"Actualizando MongoDB para el _id: {_id}")  # Mensaje de depuración
        coleccion.update_one(
            {"_id": _id},
            {"$set": nuevos_datos}
        )
        print(f"Documento con _id {_id} actualizado exitosamente.")
    except PyMongoError as e:
        print(f"Error actualizando MongoDB para el _id { _id }: {e}")

# Función para extraer los datos específicos de la respuesta de la API
def extraer_datos_especificos(datos):
    """
    Extrae los datos específicos del JSON de respuesta de la API.
    """
    data = datos.get("data", {})

    # Extraemos los valores requeridos
    resultado = {
        "retail_price_us": data.get("retail_price_us"),
        "current_value_new": data.get("current_value_new"),
        "forecast_value_new_2_years": data.get("forecast_value_new_2_years"),
        "forecast_value_new_5_years": data.get("forecast_value_new_5_years"),
        "rolling_growth_lastyear": data.get("rolling_growth_lastyear"),
        "rolling_growth_12months": data.get("rolling_growth_12months"),
        "currency": data.get("currency"),
        "set_number": data.get("set_number"),
        "current_value_used": data.get("current_value_used"),
        "price_events_new": [{
            "price_type": "new",
            "price_date": event.get("date"),
            "price_value": event.get("value")
        } for event in data.get("price_events_new", [])],
        "price_events_used": [{
            "price_type": "used",
            "price_date": event.get("date"),
            "price_value": event.get("value")
        } for event in data.get("price_events_used", [])]
    }

    return resultado

# Función principal para leer la base de datos, hacer peticiones y actualizar MongoDB
def procesar_datos():
    # Obtenemos todos los documentos que contienen la columna 'Number'
    documentos = coleccion.find({"Number": {"$exists": True}})  # Solo documentos con 'Number'
    documentos_lista = list(documentos)  # Convertir el cursor a lista
    print(f"Documentos encontrados: {len(documentos_lista)}")  # Mensaje de depuración

    datos_completos = []
    batch_size = 500  # Tamaño de los lotes (500 por petición)
    lote_actual = []

    for doc in documentos_lista:
        # Añadimos el 'Number' al lote
        lote_actual.append(doc['Number'])

        # Si el lote está lleno, procesamos el lote
        if len(lote_actual) >= batch_size:
            print(f"Procesando lote de {len(lote_actual)} elementos...")

            # Obtener los datos desde la API para el lote actual
            for Number in lote_actual:
                datos = obtener_datos(Number)
                if datos:
                    nuevos_datos = extraer_datos_especificos(datos)
                    actualizar_en_mongo(doc['_id'], nuevos_datos)  # Actualizamos el documento en MongoDB

            # Limpiamos el lote y esperamos un poco entre lotes
            lote_actual.clear()
            time.sleep(3)  # Ajusta el tiempo de espera si es necesario

    # Si hay elementos que no llegaron a llenar un lote completo, los procesamos
    if lote_actual:
        print(f"Procesando lote final de {len(lote_actual)} elementos...")
        for Number in lote_actual:
            datos = obtener_datos(Number)
            if datos:
                nuevos_datos = extraer_datos_especificos(datos)
                # No olvides actualizar los documentos restantes
                actualizar_en_mongo(doc['_id'], nuevos_datos)

    print("Proceso completado.")

# Ejecutamos el proceso
procesar_datos()


Documentos encontrados: 7679
Procesando lote de 500 elementos...
Obteniendo datos para el set: 28
Error al obtener datos para el número 28: 400
Obteniendo datos para el set: 41
Error al obtener datos para el número 41: 400
Obteniendo datos para el set: 45
Error al obtener datos para el número 45: 400
Obteniendo datos para el set: 508
Actualizando MongoDB para el _id: 67c8a69e30a1f61f23a23b00
Documento con _id 67c8a69e30a1f61f23a23b00 actualizado exitosamente.
Obteniendo datos para el set: 522
Actualizando MongoDB para el _id: 67c8a69e30a1f61f23a23b00
Documento con _id 67c8a69e30a1f61f23a23b00 actualizado exitosamente.
Obteniendo datos para el set: 523
Actualizando MongoDB para el _id: 67c8a69e30a1f61f23a23b00
Documento con _id 67c8a69e30a1f61f23a23b00 actualizado exitosamente.
Obteniendo datos para el set: 528
Actualizando MongoDB para el _id: 67c8a69e30a1f61f23a23b00
Documento con _id 67c8a69e30a1f61f23a23b00 actualizado exitosamente.
Obteniendo datos para el set: 533
Actualizando Mon

In [ ]:
import os
import requests
from pymongo import MongoClient
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Configuración de MongoDB
MONGO_URI = os.getenv("MONGO_URI")
MONGO_DB = os.getenv("MONGO_DB")
MONGO_COLECCION = os.getenv("MONGO_COLECCION")
API_KEY = os.getenv("API_KEY")

# Configuración de encabezados para la API
HEADERS = {
    "Accept": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "x-apikey": API_KEY
}

# Conectar a MongoDB
client = MongoClient(MONGO_URI)
db = client[MONGO_DB]
coleccion = db[MONGO_COLECCION]

def obtener_datos(Number):
    url = f"https://www.brickeconomy.com/api/v1/set/{Number}-1"
    print(f"📡 URL generada para la API: {url}")  # ✅ Muestra la URL antes de llamar a la API

    response = requests.get(url, headers=HEADERS)

    print(f"🔄 Código de respuesta de la API: {response.status_code}")  # ✅ Muestra el código HTTP

    if response.status_code == 200:
        print("✅ Respuesta obtenida correctamente.")
        return response.json()
    else:
        print(f"⚠ Error al obtener datos para {Number}-1: {response.status_code}")
        print(f"🔎 Respuesta de la API: {response.text}")  # ✅ Muestra el error detallado
        return None


# Extraer datos relevantes de la respuesta de la API
def extraer_datos_especificos(datos):
    data = datos.get("data", {})

    resultado = {
        "retail_price_us": data.get("retail_price_us"),
        "current_value_new": data.get("current_value_new"),
        "forecast_value_new_2_years": data.get("forecast_value_new_2_years"),
        "forecast_value_new_5_years": data.get("forecast_value_new_5_years"),
        "rolling_growth_lastyear": data.get("rolling_growth_lastyear"),
        "rolling_growth_12months": data.get("rolling_growth_12months"),
        "currency": data.get("currency"),
        "set_number": data.get("set_number"),
        "current_value_used": data.get("current_value_used"),
        "price_events_new": [  # Lista de eventos de precios nuevos
            {
                "price_type": "new",
                "price_date": event.get("date"),
                "price_value": event.get("value"),
            }
            for event in data.get("price_events_new", [])
        ],
        "price_events_used": [  # Lista de eventos de precios usados
            {
                "price_type": "used",
                "price_date": event.get("date"),
                "price_value": event.get("value"),
            }
            for event in data.get("price_events_used", [])
        ],
    }

    return resultado

# Función principal para probar el set 523-1 usando ImgFilename
def probar_set_523_1():
    Image_filename = "523-1"  # Buscamos exactamente este ImgFilename

    documento = coleccion.find_one({"ImageFilename": Image_filename})  # Buscar usando ImgFilename

    if not documento:
        print(f"No se encontró el documento con 'ImgFilename' = {Image_filename} en la base de datos.")
        return

    print(f"Probando con el set cuyo ImgFilename es: {Image_filename}")

    # Extraer solo el número antes del "-1" (ejemplo: "523-1" → "523")
    Number = Image_filename.split("-")[0]

    datos = obtener_datos(Number)
    if datos:
        nuevos_datos = extraer_datos_especificos(datos)
        print("\n🔹 Datos obtenidos de la API:\n")
        print(nuevos_datos)  # Muestra los datos en pantalla
    else:
        print("No se obtuvieron datos de la API.")

# Ejecutar la prueba
probar_set_523_1()


Probando con el set cuyo ImgFilename es: 523-1
📡 URL generada para la API: https://www.brickeconomy.com/api/v1/set/523-1
🔄 Código de respuesta de la API: 429
⚠ Error al obtener datos para 523-1: 429
🔎 Respuesta de la API: The custom error module does not recognize this error.
No se obtuvieron datos de la API.


In [1]:
import requests
import os
from dotenv import load_dotenv

# Configuración del entorno
load_dotenv()
API_KEY = os.getenv("BRICKECONOMY_API_KEY")
BASE_URL = "https://www.brickeconomy.com/api/v1/set/"
HEADERS = {
    "Accept": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "x-apikey": API_KEY
}

# Set específico para test
set_number = "523"
url = f"{BASE_URL}{set_number}"

try:
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    data = response.json().get("data", {})

    if data:
        print("\n--- Datos del Set ---")
        print(f"Número: {data.get('set_number', 'N/A')}")
        print(f"Nombre: {data.get('name', 'N/A')}")
        print(f"Tema: {data.get('theme', 'N/A')}")
        print(f"Año: {data.get('year', 'N/A')}")
        print(f"Piezas: {data.get('pieces_count', 'N/A')}")
        print(f"Minifigs: {data.get('minifigs_count', 'N/A')}")
        print(f"Precio original (USD): {data.get('retail_price_us', 'N/A')}")
        print(f"Valor actual nuevo: {data.get('current_value_new', 'N/A')} USD")
        print(f"Valor actual usado: {data.get('current_value_used', 'N/A')} USD")
        print(f"Predicción 2 años (Nuevo): {data.get('forecast_value_new_2_years', 'N/A')} USD")
        print(f"Predicción 5 años (Nuevo): {data.get('forecast_value_new_5_years', 'N/A')} USD")
        print(f"Crecimiento último año: {data.get('rolling_growth_lastyear', 'N/A')}")
        print(f"Crecimiento 12 meses: {data.get('rolling_growth_12months', 'N/A')}")
        print(f"Moneda: {data.get('currency', 'N/A')}")
        print(f"URL: {url}")
    else:
        print("No se encontraron datos para el set especificado.")

except requests.exceptions.RequestException as e:
    print(f"Error al obtener datos del set {set_number}: {e}")


Error al obtener datos del set 523: 401 Client Error: Unauthorized for url: https://www.brickeconomy.com/api/v1/set/523
